<a href="https://colab.research.google.com/github/Murphyx2/TarotReadingT5Model/blob/main/TarotCardReading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in ca

#Loading the model and Tokenizer

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


#Loading the dataset
We are using a dataset with question and answer often ask related to tarot cards from hugging face

In [ ]:
import pandas as pd

df = pd.read_parquet("hf://datasets/tarotscientist/tarot_readings_1000/data/train-00000-of-00001.parquet")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


#Exploring the dataset

In [ ]:
df.head(1)

,question,answer
0,### Question:\nWhat is the meaning of this thr...,This tarot reading indicates that you are read...


In [ ]:
#Extracting the questions and the answers
input_texts = df["question"].tolist()
target_texts = df["answer"].tolist()

# Tokenization of the inputs and targets

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

#Getting the model
model = T5ForConditionalGeneration.from_pretrained('google-t5/t5-base')

#Getting the tokenizer
tokenizer = T5Tokenizer.from_pretrained('google-t5/t5-base')

# Tokenize the input and target texts
tokenized = tokenizer(input_texts, text_target=target_texts, return_tensors="pt", padding=True, truncation=True, max_length=1024)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Testing the base model

In [ ]:
inputs = tokenizer("### Question: What is the meaning of this three card reading: The Fool, The Six of Cups, The Tower? ### Answer:", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

False


In [ ]:
from datasets import Dataset, DatasetDict

dataset = Dataset.from_dict({
    'input_ids': tokenized['input_ids'],
    'attention_mask': tokenized['attention_mask'],
    'labels': tokenized['labels']
})

# Split into train and validation sets
train_val_dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=17)
train_val_dataset = DatasetDict({
    'train': train_val_dataset['train'],
    'validation': train_val_dataset['test']
})

In [ ]:
from transformers import Trainer, TrainingArguments

# Setup training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_val_dataset["train"],
    eval_dataset=train_val_dataset["validation"],
    tokenizer=tokenizer,
)

trainer.train()

Step,Training Loss
10,11.404500
20,11.128400
30,10.041800
40,8.924100
50,7.758300
60,6.433800
70,5.221000
80,3.935300
90,3.403800
100,2.950700


TrainOutput(global_step=1000, training_loss=1.812849485397339, metrics={'train_runtime': 570.4801, 'train_samples_per_second': 14.023, 'train_steps_per_second': 1.753, 'total_flos': 409143582720000.0, 'train_loss': 1.812849485397339, 'epoch': 10.0})

In [ ]:
# Assuming 'trainer' is your Trainer object
trainer.save_model("./tarot_reading_model")

# Save the tokenizer
tokenizer.save_pretrained("./tarot_reading_model_tokenizer")

('./tarot_reading_model_tokenizer/tokenizer_config.json',
 './tarot_reading_model_tokenizer/special_tokens_map.json',
 './tarot_reading_model_tokenizer/spiece.model',
 './tarot_reading_model_tokenizer/added_tokens.json')

#Evaluation of the model

In [ ]:
# Prepare your evaluation dataset
eval_dataset = train_val_dataset["validation"]

# Evaluate
results = trainer.evaluate(eval_dataset)

# Print results
print(results)

{'eval_loss': 1.0359224081039429, 'eval_runtime': 3.7679, 'eval_samples_per_second': 53.08, 'eval_steps_per_second': 6.635, 'epoch': 10.0}


In [ ]:
# Load the model
model = T5ForConditionalGeneration.from_pretrained("./tarot_reading_model")

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained("./tarot_reading_model_tokenizer")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
inputs = tokenizer("### Question: What is the meaning of this three card reading: Six of Wands, Eight of Swords, Ten of Wands? ### Answer: ", return_tensors="pt", max_length=1024)
outputs = model.generate(
    **inputs,
    max_length=1024,
    min_length=100,   # Ensure output isn't too short
    do_sample=True,   # Use sampling for diversity
    top_k=50,         # Consider top 50 tokens
    top_p=0.95,       # Nucleus sampling with 95% probability mass
    temperature=0.7,  # Slightly more randomness in output
    num_return_sequences=3,  # Generate 3 different sequences
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


The Six of Wands suggests that you are in a position of opportunity and creativity, where you are able to take on a challenging task with new ideas and new opportunities. The Eight of Swords indicates that you may be feeling overwhelmed and unable to move forward in your life, but you will soon be able to find a way to move forward. The Ten of Wands is a card of creativity and creativity, showing that you have the power to create something new and exciting. This combination suggests that you will need to take some time to reflect on your choices and the challenges that lie ahead.


In [ ]:
#Saving model and tokenizer in zip
!zip -r tarot_reading_model.zip tarot_reading_model
#Saving model and tokenizer in zip
!zip -r tarot_reading_model_tokenizer.zip tarot_reading_model_tokenizer

  adding: tarot_reading_model/ (stored 0%)
  adding: tarot_reading_model/spiece.model (deflated 48%)
  adding: tarot_reading_model/model.safetensors (deflated 9%)
  adding: tarot_reading_model/training_args.bin (deflated 51%)
  adding: tarot_reading_model/added_tokens.json (deflated 83%)
  adding: tarot_reading_model/tokenizer_config.json (deflated 94%)
  adding: tarot_reading_model/special_tokens_map.json (deflated 85%)
  adding: tarot_reading_model/generation_config.json (deflated 30%)
  adding: tarot_reading_model/config.json (deflated 62%)
  adding: tarot_reading_model_tokenizer/ (stored 0%)
  adding: tarot_reading_model_tokenizer/spiece.model (deflated 48%)
  adding: tarot_reading_model_tokenizer/added_tokens.json (deflated 83%)
  adding: tarot_reading_model_tokenizer/tokenizer_config.json (deflated 94%)
  adding: tarot_reading_model_tokenizer/special_tokens_map.json (deflated 85%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp tarot_reading_model.zip /content/drive/MyDrive/
!cp tarot_reading_model_tokenizer.zip /content/drive/MyDrive/